In [ ]:
import pandas as pd
import plotly.express as px
import panel as pn
import ipywidgets as widgets
from datetime import datetime
import numpy as np
pn.extension("plotly", sizing_mode="stretch_width",)

In [ ]:
df = pd.read_csv("data/sales.csv", dtype={"CustomerNo": "str"})
df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%Y")
# df["Date"] = df["Date"].apply(lambda x: datetime.strptime(x, "%m/%d/%Y"))
df["Sales"] = df["Price"] * df["Quantity"]
df.sample(10)

In [ ]:
def kpi():
    sum_sales = df["Sales"].sum()
    avg_sales = df["Sales"].mean()
    sum_num_items = df["Quantity"].sum()

    return pn.Row(
        pn.Column(
            pn.pane.Markdown(f"## {sum_sales}"),
            pn.pane.Markdown("Total Sales"),
        ),
        pn.Column(
            pn.pane.Markdown(f"## {avg_sales}"),
            pn.pane.Markdown("Average amount per transaction"),
        ),
        pn.Column(
            pn.pane.Markdown(f"## {sum_num_items}"),
            pn.pane.Markdown("Total products sold"),
        ),
    )


kpi()

In [ ]:
def sales_by_time_line():
    df2 = df[["Date", "Sales"]].copy()
    df2 = df2.groupby("Date").sum().reset_index()
    fig = px.line(
        df2,
        x="Date",
        y="Sales",
        # width=1600,
    )
    fig.update_layout(hovermode="x unified")
    return fig


pn.Column(
    # pn.pane.Markdown("##Sales over time"),
    "##Sales over time",
    sales_by_time_line(),
)

In [ ]:
# from IPython.core.display import DisplayObject
date_slider = widgets.SelectionSlider(
    description="Date",
    options=[
        (pd.to_datetime(str(date)).strftime("%d %B %Y"), date)
        for date in df["Date"].sort_values().unique()
    ],
    layout={"width": "auto"},

)
# @widgets.interact(date=date_slider)
def country_sales_map():
    def create_graph(date):
        parsed_date = pd.Timestamp(date)
        df2 = df[["Country", "Sales", "Date"]].copy()
        df2 = df2.query("Date == @date")
        df2 = df2.groupby(["Country"]).sum().reset_index()
        fig = px.choropleth(
            df2,
            color="Sales",
            locations="Country",
            locationmode="country names",
            height=600,
        )
        return fig
    # date_slider = pn.widgets.DiscreteSlider(name="Date", options = {pd.to_datetime(number):number for number in df["Date"].sort_values().unique().tolist()})
    date_slider = pn.widgets.DateSlider(name="Date", start=df["Date"].min().date(), end=df["Date"].max().date(), value=df["Date"].min().date())

    return pn.Column(
        "##Sales by country for given year",
        date_slider,
        pn.bind(create_graph, date_slider)
    )

country_sales_map()

In [ ]:

def customer_by_sales():
    def create_graph(limit:int, ascending:bool):
        df2 = df[["CustomerNo", "Sales"]].copy().groupby("CustomerNo").sum().reset_index()
        df2 = df2.sort_values("Sales", ascending=ascending).head(limit)
        fig = px.bar(df2, x="CustomerNo", y="Sales")
        return fig
    limit_customer = pn.widgets.IntSlider(name="Limit", value=10, start=5, end=50)
    ascending_customer = pn.widgets.Checkbox(name="Ascending", value=False)
    return pn.Column(
        "##Customer by sales",
        limit_customer,
        ascending_customer,
        pn.bind(create_graph, limit_customer, ascending_customer)
    )

In [ ]:
# limit_product = widgets.IntSlider(value=10, min=5, max=50, layout={"width":"auto"})
# ascending_product = widgets.Checkbox(value=False)


def product_by_sales():
    def create_graph(limit:int, ascending:bool):
        df2 = df[["ProductName", "Sales"]].copy().groupby("ProductName").sum().reset_index()
        df2 = df2.sort_values("Sales", ascending=ascending).head(limit)
        fig = px.bar(df2, x="ProductName", y="Sales")
        return fig


    limit_product = pn.widgets.IntSlider(name="Limit", value=10, start=5, end=50)
    ascending_product = pn.widgets.Checkbox(name="Ascending", value=False)
    return pn.Column(
        "##Product by sales",
        limit_product,
        ascending_product,
    pn.bind(create_graph, limit=limit_product, ascending=ascending_product)
    )

In [ ]:
pn.Row(
    customer_by_sales(),
    product_by_sales(),
)